In [93]:
# Isaac G Callison
# 6350-001
# Project 3
# Due 2/13/2020

In [104]:
# imports
import random
import nltk
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk import classify
from nltk.metrics import *
import collections
from nltk.tokenize import word_tokenize
import heapq
# pull in Sotu from file 
sotu_file = 'sotu.txt'
repub_list = [] ; dem_list = []

In [105]:
# Presdents lists
r_list = ["Abraham Lincoln","Ulysses S. Grant","Rutherford B. Hayes","James Garfield","Chester a. Arthur","Benjamin Harrison","William McKinley",
    "Theodore Roosevelt","William h. Taft",
    "Warren Harding","Calvin Coolidge","Herbert Hoover","Dwight d. Eisenhower","Richard Nixon","Gerald r. Ford","Ronald Reagan","George h.w. Bush",
    "George W. Bush","Donald Trump"]
repub_list = [x.lower() for x in r_list]
d_list = ["Andrew Jackson","Martin Van Buren","james polk","Franklin Pierce","James Buchanan","Grover Cleveland","Woodrow Wilson",
    "Franklin D. Roosevelt","Harry s. Truman","John F. Kennedy","Lyndon b. Johnson","Jimmy Carter","william j. Clinton","Barack Obama"]
dem_list = [y.lower() for y in d_list]
#########################################################################################################################
## Use an intermediate test file to strip out some non-essential words
testFile = open("test.txt","w+")
stop_words = set(stopwords.words('english'))
## Open state of the union file and write out to test file with some non-essential characters stripped out. 
with open(sotu_file) as sf:
    lines = sf.readlines()
    glob = [x.replace('\'', ' ') for x in lines]
    glob = [x.lstrip('*') for x in glob]
    glob = [y.replace('\n', ' ') for y in glob]
    glob = [y.replace('\.', ' ') for y in glob]
    for line in glob:
        testFile.write(line.lower())    
testFile.close()
## Create seperate republican and democrat files to write SOTUs out to.
r_file = open("rFile.txt", 'w+')
d_file = open("dFile.txt", 'w+')
f_file = open("f_file.txt", 'w+')

r_docs = []
d_docs = []
all_words = []
## Open test file, readlines, split on '@', write out name of president and associated speech
with open("test.txt") as tf:
    new_list = []
    lines = tf.readlines()
    lines = str(lines)
    lines = lines.split('@')
    for r,item in enumerate(lines):
        s = item.split()
        for w in s:
            all_words.append(w)
        if item in repub_list:
            r_file.write(lines[r])
            r_file.write('\n')
            r_file.write(lines[r+2])
            r_docs.append(lines[r+2])
            r_file.write('\n')
        elif item in dem_list:
            d_file.write(lines[r])
            d_file.write('\n')
            d_file.write(lines[r+2])
            d_docs.append(lines[r+2])
            d_file.write('\n')

r_file.close()
d_file.close()

In [106]:
## variable for highest occuring n words. Use in nlargest for republicans and dems. 
N_LARGE = 50

## Gets highest frequency of words in the republican speeches 
r_set = set()
r_dict = {}
with open("rFile.txt") as rf:
    r_lines = rf.readlines()
    for word in r_lines:
        word = word.split()
        for item in word:
            if item not in stop_words:
                item = item.lower()
                r_dict[item] = r_dict.get(item,0)+1 
                r_set.add(item)       
r_freq = heapq.nlargest(N_LARGE, r_dict, key=r_dict.get)

In [107]:
## Gets highest frequency of words in the Democrat speeches 
d_set = set()
d_dict = {}
with open("dFile.txt") as df:
    d_lines = df.readlines()
    for word in d_lines:
        word = word.split()
        for item in word:
            if item not in stop_words:
                item = item.lower()
                d_set.add(item)
                d_dict[item] = d_dict.get(item,0)+1

d_freq = heapq.nlargest(N_LARGE, d_dict, key=d_dict.get)

In [108]:
### This is a union of the words that occur most often in BOTH republican and democrat speeches.
## The idea is that if I pull out the words that both parties use, it will reveal the true differences 
## in speech between the parties. 

common_words = (set(d_freq) & set(r_freq))

# Here are all the words with the common words in both sets of speeches removed.
#all_words_freq = nltk.FreqDist(w.lower() for w in all_words if w not in common_words)
all_words_freq = nltk.FreqDist(w.lower() for w in all_words)

## leng of all words
test_set_freq = len(all_words_freq.keys())
#######################################################
## percentage of all words to use in the word features. 
perc = 0.25
######################################################
word_features = list(all_words_freq.keys())[:(int(test_set_freq*perc))]

In [109]:
## Create tuples from both sets of words, add label. 
tup_d = []
for item in d_docs:
    #i = item.split()
    i = word_tokenize(item)
    it = (i,"democrat")
    tup_d.append(it)

tup_rep = []
for item in r_docs:
    i = word_tokenize(item)
    it = (i,"republican")
    tup_rep.append(it)

## Finally, just add all the democrat tuples to the republican tuples. 
for item in tup_d:
    tup_rep.append(item)
random.shuffle(tup_rep)
#print(tup_rep[0]) (list of words,label)

In [110]:
#################################################################################################################
############################## THIS IS THE MAIN FEATURES FUNCTION ###############################################
def find_features(sotu):
    features = {}
    wrds = set(sotu)
    for w in word_features:
        features[w] = (w in wrds)
    ## I guess I thought republicans would use shorter words, just a hunch. Also, if a word was found in high freq
    ## among republicans or democrats it was set to true
#     for item in wrds:
#         features[item] = True if len(item) <= 8 else False
#         if item in r_freq:
#             features[item] = True
#         elif item in d_freq:
#             features[item] = True 

    return features        
##################################################################################################################
##################################################################################################################

In [111]:
# construct a list consisting of the gender feature dictionaries: last letter of and the gender of a name
def find_accuracy():
    # Create featuresets and shuffle
    featuresets = [(find_features(tup_r),r) for (tup_r, r) in tup_rep ]
    random.shuffle(featuresets)

    # training/testing is set at an 80/20 split
    train_len = int(len(featuresets) * .80)
    test_len = (len(featuresets) - train_len)
    train_set, test_set = featuresets[train_len:], featuresets[:test_len]

    # Train the classifer
    classifier = NaiveBayesClassifier.train(train_set)
    
    ############ UNCOMMENT TO PRINT INFORMATIVE FEATURES ####################
    print(classifier.show_most_informative_features(10))
    
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    
    for i, (feats, label) in enumerate(test_set):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)

    # run the entire test set, and print the classifier's accuracy
    accuracy = (classify.accuracy(classifier, test_set))
    ## HEre are all the measures. 
    print("Accuracy", acc)
    r_prec =(precision(refsets['republican'], testsets['republican']))
    r_recall = (recall(refsets['republican'], testsets['republican']))
    r_measure = (f_measure(refsets['republican'], testsets['republican']))
    d_prec = (precision(refsets['democrat'], testsets['democrat']))
    d_recall = (recall(refsets['democrat'], testsets['democrat']))
    d_measure = (f_measure(refsets['democrat'], testsets['democrat']))
    return(accuracy, r_prec, r_recall, r_measure, d_prec, d_recall, d_measure)

In [112]:
## Function takes a list of lists and performs operations, averages, returns average over any number of runs. 
def return_percentages(list):
    new_list = []
    sum = 0
    for item in list:
        for num in item:
            sum += float(num)
        result = sum/len(item)
        new_list.append(result)
        sum = 0 ; result = 0
    return new_list
################################################################################################
################################## Change to run some N-number of times #########################            
TIMES_TO_RUN = 3
################################################################################################
#################################################################################################
acc_list = [] ; r_prec = [] ; d_prec = [] ; r_recall = []; d_recall = [] ;d_measure = []; r_measure = []
list_ofLists = []
for x in range(0,TIMES_TO_RUN):
    acc, r_p, r_r, r_m, d_p, d_r, d_m = find_accuracy()
    acc_list.append(acc)
    r_prec.append(r_p)
    d_prec.append(d_p)
    r_recall.append(r_r)
    d_recall.append(d_r)
    r_measure.append(r_m)
    d_measure.append(d_m)

Most Informative Features
                  became = True           democr : republ =      9.5 : 1.0
                    safe = False          republ : democr =      7.9 : 1.0
                promised = True           democr : republ =      7.4 : 1.0
                   minds = True           democr : republ =      6.7 : 1.0
                   midst = True           democr : republ =      6.7 : 1.0
                transfer = True           democr : republ =      6.7 : 1.0
              controlled = True           democr : republ =      6.7 : 1.0
                 burdens = True           democr : republ =      6.7 : 1.0
                    hand = False          republ : democr =      6.7 : 1.0
                    half = False          republ : democr =      6.0 : 1.0
None
Accuracy 1.0
Most Informative Features
                vigorous = True           republ : democr =      7.4 : 1.0
                becoming = True           republ : democr =      6.6 : 1.0
               incentive = Tru

In [113]:
list_ofLists.append(acc_list)
list_ofLists.append(r_prec)
list_ofLists.append(d_prec)
list_ofLists.append(r_recall)
list_ofLists.append(d_recall)
list_ofLists.append(r_measure)
list_ofLists.append(d_measure)
############## Print out the various measures averaged over n-number of runs. #######################
p_acc, p_r_prec, p_d_prec, p_r_rec, p_d_rec, p_r_meas, p_d_meas = return_percentages(list_ofLists)
print("Accuracy: ", p_acc)
print("Accuracy", acc)
print("Republican Precision ", p_r_prec)
print("Republican Recall ", p_r_rec)
print("Republican F-Measure ", p_r_meas)
print("Democrat Precision ", p_d_prec)
print("Democrat Recall ", p_d_rec)
print("Democrat F-Measure ", p_d_meas)

Accuracy:  0.6216216216216216
Accuracy 0.6756756756756757
Republican Precision  0.668888888888889
Republican Recall  0.7057416267942583
Republican F-Measure  0.6692510692510693
Democrat Precision  0.5984848484848485
Democrat Recall  0.5481481481481482
Democrat F-Measure  0.5491767629698665




1. Explain what you did to preprocess the data, addressing what you tried that didn’t work, and state why you
decided on the final approach you chose (include aspects in processing requirements: punctuation, text case, …)

ANSWER: I tried pulling in and parsing the presidents list, but there were too many differences between the names in the list and the names in the speeches. So I hardcoded the president names. I created an intermediate test file and wrote out the names of the presidents and the speeches in to separate files. I also removed asterisks, apostrophes, and newline characters. I created dictionaries of the most frequent words in each set of speeches, then created a set of common words in both speeches to remove. The idea is that common words among both sets of speeches would not assist in differentiation.    

2. Discuss the features you chose and why: How did you arrive at them? What did you try that did not work?

ANSWER: I had a real hard time finding a decent feature to use. I ended up creating a list off allwords and taking a set of the most common. Then I iterated through each word in each labeled speech and assigned a boolean to each word based on whether or not it was in those most common words. It works ok, sometimes the accuracy is very high. I also used length of words. My unfamiliarity with the Jupyter IDE, nltk, nltk methods, features, featuresets, etc. limited me.

3. Use the output of the show_most_informative_features method of the classifier (the data is on a per run
basis) and generalize why the top 5 features (over all 30 trials) worked as well as they did, given the input data.

ANSWER: This was the best run using word frequency and word length over 30 runs:

```
    Most Informative Features
                 capital = False          democr : republ =     10.0 : 1.0
             government, = False          democr : republ =      8.7 : 1.0
                citizens = False          democr : republ =      8.7 : 1.0
                  fought = True           democr : republ =      8.7 : 1.0
              protection = False          democr : republ =      7.6 : 1.0
             republicans = True           democr : republ =      7.3 : 1.0
                   thank = 'Rep'          democr : republ =      7.3 : 1.0
                  matter = False          democr : republ =      7.3 : 1.0
                    talk = 'Rep'          democr : republ =      7.3 : 1.0
                 things. = True           democr : republ =      7.3 : 1.0
```
    
4. What are the possible weaknesses of your approach? Is it likely that independence was violated? Is it
possible that there’s double counting in your approach? Identify where. 


ANSWER: Given that I only figured out how to utilize one featureset that I KNOW worked, I think I had no choice but to accept any weaknesses it may have had. I added length of the document but I don't know if that had any effect. I am not sure what double counting refers to.  
